In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2023-01-24 18:50:15.879403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 18:50:16.006996: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 18:50:16.007016: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-24 18:50:16.682847: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

2023-01-24 18:50:17.838076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-24 18:50:17.838130: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-24 18:50:17.838152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (578bdc85cef6): /proc/driver/nvidia/version does not exist
2023-01-24 18:50:17.838391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_history = model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 5s 5ms/step - loss: 0.3657 - sparse_categorical_accuracy: 0.9001 - val_loss: 0.1979 - val_sparse_categorical_accuracy: 0.9434
Epoch 2/6
469/469 [==============================] - 2s 3ms/step - loss: 0.1671 - sparse_categorical_accuracy: 0.9524 - val_loss: 0.1474 - val_sparse_categorical_accuracy: 0.9585
Epoch 3/6
469/469 [==============================] - 1s 3ms/step - loss: 0.1200 - sparse_categorical_accuracy: 0.9661 - val_loss: 0.1094 - val_sparse_categorical_accuracy: 0.9658
Epoch 4/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0927 - sparse_categorical_accuracy: 0.9738 - val_loss: 0.0961 - val_sparse_categorical_accuracy: 0.9722
Epoch 5/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0736 - sparse_categorical_accuracy: 0.9791 - val_loss: 0.0859 - val_sparse_categorical_accuracy: 0.9727
Epoch 6/6
469/469 [==============================] - 1s 3ms/step - loss: 0.0605 - sparse_categorical_accu

In [8]:
model_history.history

{'loss': [0.365714967250824,
  0.16710001230239868,
  0.12000691890716553,
  0.09265373647212982,
  0.0735512524843216,
  0.06045946106314659],
 'sparse_categorical_accuracy': [0.9000833630561829,
  0.9524499773979187,
  0.9661333560943604,
  0.9737833142280579,
  0.9790666699409485,
  0.9826666712760925],
 'val_loss': [0.19788549840450287,
  0.14737755060195923,
  0.10943659394979477,
  0.09609637409448624,
  0.08590958267450333,
  0.07626868039369583],
 'val_sparse_categorical_accuracy': [0.9434000253677368,
  0.9585000276565552,
  0.9657999873161316,
  0.9721999764442444,
  0.9726999998092651,
  0.9775999784469604]}

In [12]:
import pandas as pd
pd.DataFrame.from_dict(model_history.history)

,loss,sparse_categorical_accuracy,val_loss,val_sparse_categorical_accuracy
0,0.365715,0.900083,0.197885,0.9434
1,0.167100,0.952450,0.147378,0.9585
2,0.120007,0.966133,0.109437,0.9658
3,0.092654,0.973783,0.096096,0.9722
4,0.073551,0.979067,0.085910,0.9727
5,0.060459,0.982667,0.076269,0.9776


In [11]:
type(model_history.history["loss"][0])

float